In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton
from pprint import pprint
import sys
import time
import response_manager
import pandas as pd


bot = telepot.Bot('2067569208:AAHYKIRPZJ_eKtEFAtAmcFOWGlOMyRbGsXI')

In [2]:
def on_chat_message(msg):
    # Saving useful message info
    content_type, chat_type, chat_id = telepot.glance(msg)
    pprint(msg)
    questions =  pd.read_csv("DB/questions_cleared.csv", sep=",", low_memory = False, encoding='latin-1')
    
    # Check for content type
    if content_type == 'text':
        name = msg["from"]["first_name"]
        txt = str(msg['text'])
        if '/start' == txt:
            bot.sendMessage(chat_id, 'Hello {}! I\'m StackBot.'.format(name))
        
        else:
            resp = response_manager.gen_response(txt)
            bot.sendMessage(chat_id, "Here are some results:")
            
            #bot.sendMessage(chat_id, "Domanda con maggior similiarita" + questions[])

In [3]:
def on_callback_query(msg):
    query_id, from_id, query_data = telepot.glance(msg, flavor='callback_query')
    print("Callback Query: ", query_id, from_id, query_data)
    
    bot.answerCallbackQuery(query_id, text="YEAH")
    bot.sendMessage(query_id, "What are you looking for?")

In [ ]:
# Managing different inputs
bot.message_loop({'chat': on_chat_message,
                  'callback_query': on_callback_query})

# Keeping the program running
while 1:
    time.sleep(3)

{'chat': {'first_name': 'Andrea', 'id': 235419317, 'type': 'private'},
 'date': 1635176949,
 'from': {'first_name': 'Andrea',
          'id': 235419317,
          'is_bot': False,
          'language_code': 'en'},
 'message_id': 58,
 'text': 'i have a problem wtith indentation in python'}


C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[['indentation', 0.8351992613633952], ['python', 0.45672086655420413], ['problem', 0.3063466074106518], ['projest', 0.0], ['promoted', 0.0]]

Keywords:
[('113', 0.4), ('218', 0.4), ('331', 0.4), ('1098', 0.4), ('1198', 0.4)]


Traceback (most recent call last):
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\telepot\__init__.py", line 1158, in collector
    callback(item)
  File "C:\Users\andri\.virtualenvs\Progetto_StackOverflow-UQruahqZ\lib\site-packages\telepot\helper.py", line 1041, in route
    return fn(msg, *args, **kwargs)
  File "C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_15816/4256192332.py", line 16, in on_chat_message
    bot.sendMessage(chat_id, "Here are some results:" + resp)
TypeError: can only concatenate str (not "list") to str
